# Weather Planner – Service Tests (ohne MCP)

Dieses Notebook testet die **reinen Services** aus `weather_planner/services/`.

- Geocoding (Nominatim)
- Wetter (Open-Meteo)
- POIs (Overpass)
- Scoring/Ranking (Heuristik)

Hinweise:
- Alle APIs sind **tokenfrei**, aber **rate-limited** → Cache ist standardmäßig aktiv.
- Wenn Overpass langsam ist, kann ein zweiter Run dank Cache deutlich schneller sein.


In [1]:
# Wenn du dieses Notebook im Projekt-Root ausführst, sollte der Import direkt funktionieren.
# Falls nicht (z.B. weil du das Notebook woanders öffnest), setze den Root-Pfad explizit:
import os, sys, pathlib

ROOT = pathlib.Path().resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)


Project root: /home/simon/Workshop_Agentic_AI/mcp_server_travel


In [2]:
from mcp_tools_travel.core.cache import FileCache
from mcp_tools_travel.core.schemas import Coordinates, TripSpec, WeatherDay, WeatherProfile, Spot
from mcp_tools_travel.services.geocoding import geocode_destination
from mcp_tools_travel.services.weather import get_weather_profile
from mcp_tools_travel.services.pois import get_activity_spots
from mcp_tools_travel.services.scoring import score_spots_heuristic

from mcp_tools_travel.utils.geo import haversine_km, travel_time_minutes


## 1) Cache initialisieren

Wir legen den Cache in `./data/cache` an (relativ zum Projekt-Root).

In [3]:
cache_dir = "./data/cache"
cache = FileCache(cache_dir)
cache_dir


'./data/cache'

## 2) Distance und Timecalculation testen


In [4]:
destination_A = "Barcelona"
destination_B = "Palma"

coords_A = geocode_destination(destination_A, cache=cache)
coords_B = geocode_destination(destination_B, cache=cache)

dist = haversine_km(coords_A.lat, coords_A.lon, coords_B.lat, coords_B.lon)

travel_time = travel_time_minutes(dist)

print(f"Distance destination_A:{destination_A}, destination_B:{destination_B} = {dist} km travel_time:{travel_time} minutes")

Distance destination_A:Barcelona, destination_B:Palma = 205.52677543399773 km travel_time:205.52677543399773 minutes


## 3) Geocoding testen (Nominatim)

Wir geocoden eine Zielregion. Du kannst das frei ändern (z.B. `Barcelona`, `Toskana`, `Mittelmeer`).

In [5]:
destination = "Barcelona"
coords = geocode_destination(destination, cache=cache)

if isinstance(coords, Coordinates):
    print("Coordinates bestätigt:")
else:
    print("Ergebnis hat anderen Typ:", type(coords))

print(coords)


Coordinates bestätigt:
lat=41.3825802 lon=2.177073


## 4) Wetter testen forecast (Open-Meteo bis 16 Tage)

Wir holen das tägliche Wetterprofil für einen Zeitraum.

> Tipp: Für Trainings ist es oft hilfreich, nahe Zukunftsdaten zu nehmen, um "realistische" Forecasts zu sehen.

In [6]:
from datetime import date, timedelta

start = date.today() + timedelta(days=1)
end   = start + timedelta(days=7)

weather = get_weather_profile(coords, start, end, cache=cache, include_raw=False)

if isinstance(weather, WeatherProfile):
    print("WeatherProfile bestätigt:")
else:
    print("Ergebnis hat anderen Typ:", type(weather))

weather


WeatherProfile bestätigt:


WeatherProfile(start_date=datetime.date(2026, 1, 30), end_date=datetime.date(2026, 2, 6), days=[WeatherDay(date=datetime.date(2026, 1, 30), temp_min_c=9.4, temp_max_c=15.1, precipitation_mm=0.2, wind_max_kmh=18.9, weather_code=61), WeatherDay(date=datetime.date(2026, 1, 31), temp_min_c=9.1, temp_max_c=14.5, precipitation_mm=0.9, wind_max_kmh=15.3, weather_code=61), WeatherDay(date=datetime.date(2026, 2, 1), temp_min_c=5.3, temp_max_c=11.6, precipitation_mm=0.3, wind_max_kmh=9.6, weather_code=3), WeatherDay(date=datetime.date(2026, 2, 2), temp_min_c=5.6, temp_max_c=11.7, precipitation_mm=7.8, wind_max_kmh=6.2, weather_code=61), WeatherDay(date=datetime.date(2026, 2, 3), temp_min_c=6.9, temp_max_c=14.4, precipitation_mm=0.0, wind_max_kmh=11.5, weather_code=3), WeatherDay(date=datetime.date(2026, 2, 4), temp_min_c=4.2, temp_max_c=14.2, precipitation_mm=0.0, wind_max_kmh=15.1, weather_code=3), WeatherDay(date=datetime.date(2026, 2, 5), temp_min_c=6.8, temp_max_c=14.9, precipitation_mm=0.9,

In [7]:
import pandas as pd

df_weather = pd.DataFrame([d.model_dump() for d in weather.days])
print(df_weather)

         date  temp_min_c  temp_max_c  precipitation_mm  wind_max_kmh  \
0  2026-01-30         9.4        15.1               0.2          18.9   
1  2026-01-31         9.1        14.5               0.9          15.3   
2  2026-02-01         5.3        11.6               0.3           9.6   
3  2026-02-02         5.6        11.7               7.8           6.2   
4  2026-02-03         6.9        14.4               0.0          11.5   
5  2026-02-04         4.2        14.2               0.0          15.1   
6  2026-02-05         6.8        14.9               0.9          20.5   
7  2026-02-06        10.3        16.8               0.0          25.4   

   weather_code  
0            61  
1            61  
2             3  
3            61  
4             3  
5             3  
6             3  
7             3  


### Wetter-Summary (Aggregationen)

Diese Aggregationen werden später für Scoring, Packing-List und Heuristiken genutzt.

In [8]:
{
    "temp_min_c": weather.temp_min_c,
    "temp_max_c": weather.temp_max_c,
    "precip_total_mm": weather.precip_total_mm,
    "rainy_days": weather.rainy_days,
    "wind_max_kmh": weather.wind_max_kmh,
}

{'temp_min_c': None,
 'temp_max_c': None,
 'precip_total_mm': None,
 'rainy_days': None,
 'wind_max_kmh': None}

## 5) Wetter testen historical (Open-Meteo ab 17 Tage)

Wir holen das tägliche Wetterprofil für einen Zeitraum.

In [9]:
# Far-future range: triggers historical fallback built from archive data
future_start = date(date.today().year + 1, 6, 10)
future_end   = date(date.today().year + 1, 6, 17)

weather_future = get_weather_profile(coords, future_start, future_end, cache=cache, include_raw=False)

if isinstance(weather_future, WeatherProfile):
    print("WeatherProfile bestätigt:")
else:
    print("Ergebnis hat anderen Typ:", type(weather_future))

print('source:', weather_future.source, '| estimate:', weather_future.is_estimate)
print('reference_years (sample):', weather_future.reference_years[:5])
weather_future


WeatherProfile bestätigt:
source: historical_fallback | estimate: True
reference_years (sample): [2025, 2024, 2023, 2022, 2021]


WeatherProfile(start_date=datetime.date(2027, 6, 10), end_date=datetime.date(2027, 6, 17), days=[WeatherDay(date=datetime.date(2027, 6, 10), temp_min_c=17.68, temp_max_c=25.78, precipitation_mm=0.71, wind_max_kmh=17.509999999999998, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 11), temp_min_c=18.02, temp_max_c=25.62, precipitation_mm=4.859999999999999, wind_max_kmh=16.12, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 12), temp_min_c=18.19, temp_max_c=25.56, precipitation_mm=1.5, wind_max_kmh=19.51, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 13), temp_min_c=17.95, temp_max_c=26.28, precipitation_mm=1.79, wind_max_kmh=18.68, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 14), temp_min_c=18.28, temp_max_c=26.42, precipitation_mm=1.18, wind_max_kmh=19.3, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 15), temp_min_c=18.89, temp_max_c=26.41, precipitation_mm=0.02, wind_max_kmh=17.79, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 

In [10]:
import pandas as pd

df_weather_future = pd.DataFrame([d.model_dump() for d in weather_future.days])
print(df_weather_future)


         date  temp_min_c  temp_max_c  precipitation_mm  wind_max_kmh  \
0  2027-06-10       17.68       25.78              0.71         17.51   
1  2027-06-11       18.02       25.62              4.86         16.12   
2  2027-06-12       18.19       25.56              1.50         19.51   
3  2027-06-13       17.95       26.28              1.79         18.68   
4  2027-06-14       18.28       26.42              1.18         19.30   
5  2027-06-15       18.89       26.41              0.02         17.79   
6  2027-06-16       18.71       26.60              0.48         18.88   
7  2027-06-17       19.06       25.98              0.38         18.46   

   weather_code  
0             3  
1             3  
2             3  
3             3  
4             3  
5             3  
6             3  
7             3  


### Wetter-Summary (Aggregationen)

Diese Aggregationen werden später für Scoring, Packing-List und Heuristiken genutzt.

In [11]:
{
    "temp_min_c": weather_future.temp_min_c,
    "temp_max_c": weather_future.temp_max_c,
    "precip_total_mm": weather_future.precip_total_mm,
    "rainy_days": weather_future.rainy_days,
    "wind_max_kmh": weather_future.wind_max_kmh,
}


{'temp_min_c': 17.68,
 'temp_max_c': 26.6,
 'precip_total_mm': 10.92,
 'rainy_days': 4,
 'wind_max_kmh': 19.51}

## 6) POI/Spots testen (Overpass)

Wir ziehen Spots passend zu einer Aktivität und einem Suchradius.

In [12]:
from typing import List

activity = "sightseeing"   # z.B. hiking, running, beach, sightseeing
radius_km = 10.0

spots = get_activity_spots(coords.lat, coords.lon, radius_km, activity, cache=cache)

if isinstance(spots, list):
    print("Liste bestätigt")
    if all(isinstance(s, Spot) for s in spots):
        print("und die Elemente sind Spot-Instanzen")
    else:
        print("aber die Elemente haben anderen Typ:", {type(s) for s in spots})
else:
    print("kein list:", type(spots))

len(spots), spots[:3]


Liste bestätigt
und die Elemente sind Spot-Instanzen


(75,
 [Spot(name='Portal del Bisbe', lat=41.3840055, lon=2.1754247, tags={'historic': 'city_gate', 'name': 'Portal del Bisbe', 'name:ca': 'Portal del Bisbe', 'wikidata': 'Q11942461', 'wikimedia_commons': 'Category:Portal del Bisbe (Barcelona)', 'wikipedia': 'ca:Portal del Bisbe'}, activity='sightseeing', distance_km=None, travel_time_min=None, score=None),
  Spot(name='Barcelona a Prim', lat=41.3863896, lon=2.1867952, tags={'artist_name': 'Lluís Puiggener', 'artwork_type': 'statue', 'historic': 'memorial', 'memorial': 'statue', 'name': 'Barcelona a Prim', 'name:ca': 'Barcelona a Prim', 'note': 'Tot i que l’estatua del general, obra de Lluís Puiggener, fou enderrocada durant la Guerra Civil, avui està al seu emplaçament original. A la base hi trobem dos relleus amb dues escenes militars de la vida de Prim.', 'statue': 'equestrian', 'tourism': 'artwork', 'website': 'http://paradoxesdelavida.blogspot.com.es/2014/03/iv-prim-lexposicio-universal-de-1888bcn.html', 'wikidata': 'Q18156161', 'w

In [13]:
df_spots = pd.DataFrame([{"name": s.name, "lat": s.lat, "lon": s.lon, "activity": s.activity} for s in spots])
df_spots.head(10)


,name,lat,lon,activity
0,Portal del Bisbe,41.384006,2.175425,sightseeing
1,Barcelona a Prim,41.386390,2.186795,sightseeing
2,Homenatge als castellers,41.382033,2.177009,sightseeing
3,El Cul. A Santiago Roldan,41.388947,2.193827,sightseeing
4,Farmàcia Hereus Torras Surroca,41.448540,2.248803,sightseeing
5,Elogi de l'aigua,41.418219,2.145737,sightseeing
6,Pastelería Escribá Rambla De Les Flors,41.381526,2.172671,sightseeing
7,Salvador Allende,41.439136,2.225117,sightseeing
8,Ernesto 'Che' Guevara,41.441879,2.223657,sightseeing
9,Jardins d'Ernest Lluch - Cementiri Vell,41.455902,2.209765,sightseeing


## 7) Scoring/Ranking testen

Wir berechnen Distanz/Travel-Time + Heuristik-Score und sortieren absteigend.

> Später ersetzt/ergänzt ein ML-Modell die Heuristik – aber die Schnittstelle bleibt gleich.

In [14]:
ranked = score_spots_heuristic(coords.lat, coords.lon, spots, weather)

if isinstance(spots, list):
    print("Liste bestätigt")
    if all(isinstance(s, Spot) for s in spots):
        print("und die Elemente sind Spot-Instanzen")
    else:
        print("aber die Elemente haben anderen Typ:", {type(s) for s in spots})
else:
    print("kein list:", type(spots))

ranked[:5]


Liste bestätigt
und die Elemente sind Spot-Instanzen


[Spot(name='Homenatge als castellers', lat=41.3820329, lon=2.177009, tags={'historic': 'monument', 'name': 'Homenatge als castellers', 'name:ca': 'Homenatge als castellers', 'wikidata': 'Q11926010', 'wikimedia_commons': 'Category:Homenatge als castellers', 'wikipedia': 'ca:Homenatge als castellers'}, activity='sightseeing', distance_km=0.06109078286131426, travel_time_min=0.24436313144525704, score=69.90836382570802),
 Spot(name="Temple d'August", lat=41.3833749, lon=2.1772306, tags={'addr:housenumber': '10', 'addr:postcode': '08002', 'addr:street': 'Carrer del Paradís', 'fee': 'no', 'historic': 'archaeological_site;ruins', 'name': "Temple d'August", 'name:ca': "Temple d'August", 'name:de': 'Augustus-Tempel', 'name:es': 'Templo de Augusto', 'name:fr': "Temple d'augyst", 'name:it': 'Corte antica', 'name:ru': 'Римские колонны 1в. н.э.', 'name:zh': '奧古斯都神廟', 'operator': 'MUHBA', 'tourism': 'attraction', 'wheelchair': 'limited', 'wikidata': 'Q3819019', 'wikimedia_commons': 'Category:Temple

In [15]:
df_ranked = pd.DataFrame([
    {
        "name": s.name,
        "activity": s.activity,
        "distance_km": round(s.distance_km or 0, 2),
        "travel_time_min": round(s.travel_time_min or 0, 1),
        "score": round(s.score or 0, 1),
    }
    for s in ranked[:15]
])
df_ranked


,name,activity,distance_km,travel_time_min,score
0,Homenatge als castellers,sightseeing,0.06,0.2,69.9
1,Temple d'August,sightseeing,0.09,0.4,69.9
2,Monument als Herois de 1809,sightseeing,0.15,0.6,69.8
3,Portal del Bisbe,sightseeing,0.21,0.8,69.7
4,Ramon Berenguer III el Gran,sightseeing,0.22,0.9,69.7
5,Pastelería Escribá Rambla De Les Flors,sightseeing,0.39,1.5,69.4
6,Font del Geni català,sightseeing,0.54,2.1,69.2
7,Triangle rosa,sightseeing,0.86,3.5,68.7
8,Barcelona a Prim,sightseeing,0.92,3.7,68.6
9,A Joan Güell i Ferrer,sightseeing,1.06,4.2,68.4


## 8) Mini-End-to-End: mehrere Aktivitäten

Optional: Für mehrere Aktivitäten nacheinander ziehen und die Top-Spots ansehen.


In [16]:
activities = ["sightseeing", "running"]  # z.B. ["beach","hiking","sightseeing"]
radius_km = 12.0
top_k = 5

results = {}
for act in activities:
    s = get_activity_spots(coords.lat, coords.lon, radius_km, act, cache=cache)
    r = score_spots_heuristic(coords.lat, coords.lon, s, weather)[:top_k]
    results[act] = r

for act, r in results.items():
    print("\n===", act, "===")
    for item in r:
        print(f"- {item.name} | {item.score:.1f} | {item.distance_km:.2f} km")



=== sightseeing ===
- Homenatge als castellers | 69.9 | 0.06 km
- Temple d'August | 69.9 | 0.09 km
- Monument als Herois de 1809 | 69.8 | 0.15 km
- Portal del Bisbe | 69.7 | 0.21 km
- Ramon Berenguer III el Gran | 69.7 | 0.22 km

=== running ===
- Carrer d'Arlet | 71.9 | 0.07 km
- Carrer de Salomó ben Adret | 71.8 | 0.11 km
- Carrer de la Palla | 71.7 | 0.23 km
- Carrer de n'Alsina | 71.6 | 0.23 km
- Carrer de les Heures | 71.6 | 0.25 km
